In [10]:
!pip install -q noisereduce pyloudnorm soundfile librosa transformers torch

### Wipe Kaggle's Output Storage

In [11]:
import os
import shutil

working_dir = '/kaggle/working'

print("🟡 Wiping Kaggle working directory...")

for item in os.listdir(working_dir):
    item_path = os.path.join(working_dir, item)
    try:
        if os.path.isfile(item_path) or os.path.islink(item_path):
            os.unlink(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)
    except Exception as e:
        print(f"🔴 Failed to delete {item_path}. Reason: {e}")

print("🟢 Kaggle working directory is completely clean.")

🟡 Wiping Kaggle working directory...
🟢 Kaggle working directory is completely clean.


### Wipe a Specific Folder/File

In [12]:
import shutil
from pathlib import Path

def delete_target(target_path):
    target = Path(target_path)
    
    if not target.exists():
        print(f"🟡 Target does not exist: {target}")
        return
        
    try:
        if target.is_file() or target.is_symlink():
            target.unlink()
            print(f"🟢 Successfully deleted file: {target}")
        elif target.is_dir():
            shutil.rmtree(target)
            print(f"🟢 Successfully deleted directory: {target}")
    except Exception as e:
        print(f"🔴 Failed to delete {target}. Reason: {e}")

# Just drop your path here
delete_target('/kaggle/working/your_folder_or_file_here')

🟡 Target does not exist: /kaggle/working/your_folder_or_file_here


In [13]:
# ═══════════════════════════════════════════════════════════════════════
# PHASE 0 — Environment & Path Configuration
# ═══════════════════════════════════════════════════════════════════════

import os, json, hashlib, time, random, warnings, shutil, pickle
from pathlib import Path
from typing import Optional, List, Dict, Tuple
from collections import defaultdict

import numpy as np
import librosa
import soundfile as sf
from scipy.signal import fftconvolve
import noisereduce as nr

warnings.filterwarnings('ignore')
random.seed(42)
np.random.seed(42)

# ═══════════════════════════════════════════════════════════════════════
# KAGGLE INPUT PATHS  (datasets attached to this notebook)
# ═══════════════════════════════════════════════════════════════════════
PATHS = {
    'irs':       Path('/kaggle/input/datasets/itorousa/impulse-responses'),
    'mit_raw':   Path('/kaggle/input/notebooks/itorousa/irs/mit_irs/Audio'
                      'https://mcdermottlab.mit.edu/Reverb/IRMAudio/Audio.zip'),
    'ljspeech':  Path('/kaggle/input/datasets/dromosys/ljspeech'),
    'vctk':      Path('/kaggle/input/datasets/kynthesis/vctk-corpus'
                      '/VCTK-Corpus/wav48'),
    'langid_en': Path('/kaggle/input/datasets/shrivatssudhir'
                      '/language-identifier/english/clips'),
}

# Chaining — previous run's output (attached as input to this notebook).
# Adjust the path for each subsequent run.
PREV_RUN_PATH = Path('/kaggle/input/notebooks/itorousa/genesis-data-run1')

# ═══════════════════════════════════════════════════════════════════════
# OUTPUT
# ═══════════════════════════════════════════════════════════════════════
OUTPUT          = Path('/kaggle/working')
BATCH_DIR       = OUTPUT / 'batches'
CLAP_DIR        = OUTPUT / 'clap_model'
STERILIZED_DIR  = OUTPUT / 'sterilized_batches'
MIT_IR_DIR      = OUTPUT / 'irs' / 'mit_irs'

for d in [BATCH_DIR, CLAP_DIR, STERILIZED_DIR, MIT_IR_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ═══════════════════════════════════════════════════════════════════════
# AUDIO PARAMETERS
# ═══════════════════════════════════════════════════════════════════════
SR            = 48_000
CLIP_SEC      = 5.0
CLIP_SAMPLES  = int(SR * CLIP_SEC)    # 240,000 samples

# ═══════════════════════════════════════════════════════════════════════
# BUDGET & BATCHING
# ═══════════════════════════════════════════════════════════════════════
TRIPLES_PER_BATCH  = 500
MAX_OUTPUT_GB      = 19.0             # safety margin under 20 GB cap
AUGMENTATIONS_MIN  = 3
AUGMENTATIONS_MAX  = 6

print(f"SR={SR}, CLIP_SEC={CLIP_SEC}, CLIP_SAMPLES={CLIP_SAMPLES}")
print(f"Output budget: {MAX_OUTPUT_GB} GB, batch size: {TRIPLES_PER_BATCH}")


SR=48000, CLIP_SEC=5.0, CLIP_SAMPLES=240000
Output budget: 19.0 GB, batch size: 500


In [14]:
# ═══════════════════════════════════════════════════════════════════════
# PHASE 1 — Aggressive Checkpoint Initialization & Hardware Protection
# ═══════════════════════════════════════════════════════════════════════
CHECKPOINT_PATH = OUTPUT / 'checkpoint.json'

def load_checkpoint() -> dict:
    '''Load checkpoint: previous run → current working dir → fresh start.'''
    # 1) Check previous run first (chaining)
    prev_ckpt = PREV_RUN_PATH / 'checkpoint.json'
    if prev_ckpt.exists():
        with open(prev_ckpt) as f:
            ckpt = json.load(f)
        ckpt['run_number'] += 1
        print(f"[Checkpoint] ♻ Resuming from previous run: "
              f"{ckpt['triples_completed']} triples done")
        return ckpt

    # 2) Check current working dir (kernel restart mid-session)
    if CHECKPOINT_PATH.exists():
        with open(CHECKPOINT_PATH) as f:
            return json.load(f)

    # 3) Fresh start
    return {
        'batch_id': 0,
        'triples_completed': 0,
        'vocal_cursor': 0,
        'run_number': 1,
    }

def save_checkpoint(ckpt: dict):
    with open(CHECKPOINT_PATH, 'w') as f:
        json.dump(ckpt, f, indent=2)

def get_output_size_gb() -> float:
    '''Calculate the exact size of /kaggle/working in GB.'''
    total = sum(f.stat().st_size for f in OUTPUT.rglob('*') if f.is_file())
    return total / (1024 ** 3)

ckpt = load_checkpoint()
print(f"[Checkpoint] Run #{ckpt['run_number']}, "
      f"{ckpt['triples_completed']} triples completed so far, "
      f"starting at vocal cursor {ckpt['vocal_cursor']}")


[Checkpoint] Run #1, 0 triples completed so far, starting at vocal cursor 0


In [15]:
import urllib.request
import zipfile
import shutil
import json
import librosa
import numpy as np
from typing import Dict, List, Optional
from pathlib import Path

# ═══════════════════════════════════════════════════════════════════════
# PHASE 2 — Impulse Response Acquisition & Pooling
# ═══════════════════════════════════════════════════════════════════════
CATALOGUE_PATH = OUTPUT / 'ir_catalogue.json'
MIT_RAW_URL = 'https://mcdermottlab.mit.edu/Reverb/IRMAudio/Audio.zip'

# ─── Fast path: reload from existing catalogue ───────────────────────
if CATALOGUE_PATH.exists():
    print("🟢 Found existing ir_catalogue.json — skipping audio-loading block.")
    with open(CATALOGUE_PATH, 'r') as f:
        ir_catalogue = json.load(f)

    all_irs: Dict[str, dict] = {}   # will NOT have audio yet; lazy-loaded later
    bad_pool:    List[str] = []
    target_pool: List[str] = []

    for ir_id, feats in ir_catalogue.items():
        if feats['source'] == 'mit':
            target_pool.append(ir_id)
        elif feats['rt60'] > 0.25 or feats['c50'] < 8:
            bad_pool.append(ir_id)
        else:
            bad_pool.append(ir_id)

    # Safety: ensure target pool is adequate
    if len(target_pool) < 20:
        mit_ids = [k for k, v in ir_catalogue.items() if v['source'] == 'mit']
        target_pool = mit_ids if mit_ids else list(ir_catalogue.keys())[:50]
        bad_pool = [k for k in ir_catalogue if k not in target_pool]

    print(f"🟢 Loaded {len(ir_catalogue)} IRs from catalogue")
    print(f"🟢 Bad pool: {len(bad_pool)} | Target pool: {len(target_pool)}")
    _catalogue_loaded_from_disk = True

else:
    _catalogue_loaded_from_disk = False

    # ─── Stage 1: Copy / acquire MIT IRs ─────────────────────────────
    prev_mit = PREV_RUN_PATH / 'irs' / 'mit_irs'
    MIT_IR_DIR.mkdir(parents=True, exist_ok=True)
    
    if prev_mit.exists() and not any(MIT_IR_DIR.iterdir()):
        print("🟢 [Acquire] Copying MIT IRs from previous run...")
        shutil.copytree(prev_mit, MIT_IR_DIR, dirs_exist_ok=True)
        print("🟢 [Acquire] MIT IRs staged from previous run ✓")
    elif not any(MIT_IR_DIR.iterdir()):
        print("🟢 [Acquire] Downloading MIT IRs...")
        zip_path = MIT_IR_DIR / 'mit_irs.zip'
        urllib.request.urlretrieve(MIT_RAW_URL, str(zip_path))
        with zipfile.ZipFile(zip_path, 'r') as zf:
            zf.extractall(MIT_IR_DIR)
        zip_path.unlink()
        print("🟢 [Acquire] MIT IRs downloaded and extracted ✓")
    else:
        print("🟢 [Acquire] MIT IRs already staged or unavailable")

    # ─── Stage 2: Load & normalize every IR to 48 kHz mono ───────────
    def load_and_normalize_ir(filepath: Path, target_sr: int = SR) -> Optional[np.ndarray]:
        '''Load an IR file, force mono / target SR, peak-normalize.'''
        try:
            audio, _ = librosa.load(str(filepath), sr=target_sr, mono=True)
            if len(audio) < 64:
                return None
            peak = np.max(np.abs(audio))
            if peak > 1e-6:
                audio = audio / peak
            return audio.astype(np.float32)
        except Exception:
            return None

    print("\n🟢 [Acquire] Loading all impulse responses...")
    all_irs: Dict[str, dict] = {}

    # Bad pool — user's unlabeled IRs
    for ext in ('*.irs', '*.wav'):
        for f in sorted(PATHS['irs'].rglob(ext)):
            ir = load_and_normalize_ir(f)
            if ir is not None:
                all_irs[f'user_{f.stem}'] = {'audio': ir, 'source': 'user'}

    # Target pool — MIT labeled IRs
    if MIT_IR_DIR.exists():
        for f in sorted(MIT_IR_DIR.rglob('*.wav')):
            ir = load_and_normalize_ir(f)
            if ir is not None:
                all_irs[f'mit_{f.stem}'] = {'audio': ir, 'source': 'mit'}

    print(f"🟢 [Acquire] Loaded {len(all_irs)} IRs "
          f"(user/bad: {sum(1 for v in all_irs.values() if v['source']=='user')}, "
          f"MIT/target: {sum(1 for v in all_irs.values() if v['source']=='mit')})")

    # ─── Stage 3: Classify & build pools ─────────────────────────────
    def compute_ir_features(ir_audio: np.ndarray, sr: int = SR) -> dict:
        '''Compute RT60 estimate, spectral centroid, and clarity (C50).'''
        energy = ir_audio ** 2
        cumsum = np.cumsum(energy[::-1])[::-1]

        # RT60 — time for energy to decay 60 dB
        rt60 = len(ir_audio) / sr
        if cumsum[0] > 1e-10:
            decay_db = 10 * np.log10(cumsum / cumsum[0] + 1e-12)
            idx = np.where(decay_db < -60)[0]
            if len(idx) > 0:
                rt60 = idx[0] / sr

        # Spectral centroid
        S = np.abs(np.fft.rfft(ir_audio))
        freqs = np.fft.rfftfreq(len(ir_audio), 1 / sr)
        centroid = float(np.sum(freqs * S) / (np.sum(S) + 1e-10))

        # Clarity C50 — early-to-late energy ratio at 50 ms
        split = int(0.05 * sr)
        early = np.sum(ir_audio[:split] ** 2) + 1e-12
        late  = np.sum(ir_audio[split:] ** 2) + 1e-12
        c50   = float(10 * np.log10(early / late))

        return {
            'rt60': round(rt60, 4),
            'centroid': round(centroid, 1),
            'c50': round(c50, 2),
            'length_sec': round(len(ir_audio) / sr, 4),
        }

    ir_catalogue = {}
    bad_pool:    List[str] = []
    target_pool: List[str] = []

    for ir_id, ir_data in all_irs.items():
        feats = compute_ir_features(ir_data['audio'])
        ir_data['features'] = feats
        ir_catalogue[ir_id] = {'source': ir_data['source'], **feats}

        # MIT IRs → target pool (labeled ground-truth rooms)
        # User IRs → bad pool (unlabeled/degraded)
        if ir_data['source'] == 'mit':
            target_pool.append(ir_id)
        else:
            bad_pool.append(ir_id)

    # Safety: ensure target pool is adequate
    if len(target_pool) < 20:
        mit_ids = sorted(
            [k for k, v in all_irs.items() if v['source'] == 'mit'],
            key=lambda k: all_irs[k]['features']['c50'], reverse=True
        )
        target_pool = mit_ids if mit_ids else list(all_irs.keys())[:50]
        bad_pool = [k for k in all_irs if k not in target_pool]

    print(f"🟢 [Classify] Bad pool:    {len(bad_pool)} IRs")
    print(f"🟢 [Classify] Target pool: {len(target_pool)} IRs")

    # 💾 Save catalogue
    with open(CATALOGUE_PATH, 'w') as f:
        json.dump(ir_catalogue, f, indent=2)
    print("🟢 [Classify] ir_catalogue.json saved ✓")

🟢 [Acquire] Downloading MIT IRs...
🟢 [Acquire] MIT IRs downloaded and extracted ✓

🟢 [Acquire] Loading all impulse responses...
🟢 [Acquire] Loaded 659 IRs (user/bad: 389, MIT/target: 270)
🟢 [Classify] Bad pool:    389 IRs
🟢 [Classify] Target pool: 270 IRs
🟢 [Classify] ir_catalogue.json saved ✓


In [17]:
import torch
from transformers import ClapModel, ClapProcessor

# ═══════════════════════════════════════════════════════════════════════
# PHASE 3 — CLAP Target Embedding Cache
# ═══════════════════════════════════════════════════════════════════════

CLAP_MODEL_ID = "laion/larger_clap_music_and_speech"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🟢 [CLAP] Device: {device}")

# ─── Model loading chain: local → previous run → download ────────────
prev_clap = PREV_RUN_PATH / 'clap_model'

if (CLAP_DIR / 'config.json').exists():
    print("🟢 [CLAP] Loading frozen model from current working dir...")
    clap_processor = ClapProcessor.from_pretrained(CLAP_DIR)
    clap_model = ClapModel.from_pretrained(CLAP_DIR).to(device).eval()

elif prev_clap.exists() and (prev_clap / 'config.json').exists():
    print("🟢 [CLAP] Copying frozen model from previous run...")
    shutil.copytree(prev_clap, CLAP_DIR, dirs_exist_ok=True)
    clap_processor = ClapProcessor.from_pretrained(CLAP_DIR)
    clap_model = ClapModel.from_pretrained(CLAP_DIR).to(device).eval()

else:
    print("🟢 [CLAP] Downloading model from Hugging Face...")
    clap_processor = ClapProcessor.from_pretrained(CLAP_MODEL_ID)
    clap_model = ClapModel.from_pretrained(CLAP_MODEL_ID).to(device).eval()
    clap_model.save_pretrained(CLAP_DIR)
    clap_processor.save_pretrained(CLAP_DIR)
    print(f"🟢 [CLAP] Frozen model saved to {CLAP_DIR}")

CLAP_DIM = clap_model.config.projection_dim
print(f"🟢 [CLAP] Loaded — embedding dim = {CLAP_DIM}")


def get_clap_audio_embedding(audio: np.ndarray, sr: int = SR) -> np.ndarray:
    """
    Encode audio through CLAP's audio tower. Returns (CLAP_DIM,) float32.

    ⚠ Critical: extract .pooler_output from the wrapper before calling .cpu().
    """
    inputs = clap_processor(
        audio=audio, sampling_rate=sr, return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = clap_model.get_audio_features(**inputs)
        emb = outputs.pooler_output
    return emb.cpu().numpy().flatten().astype(np.float32)


# ─── Pre-compute CLAP embeddings for every target-pool IR ─────────────
# Convolve each IR with 3s of white noise so CLAP has a rich scene.
CLAP_CACHE_PATH = OUTPUT / 'clap_cache.npz'
prev_cache = PREV_RUN_PATH / 'clap_cache.npz'

if CLAP_CACHE_PATH.exists():
    print("🟢 [CLAP] Found existing clap_cache.npz — skipping embedding compute.")
    clap_cache = dict(np.load(CLAP_CACHE_PATH))
    print(f"🟢 [CLAP] Loaded {len(clap_cache)} embeddings from cache ✓  (dim={CLAP_DIM})")

elif prev_cache.exists():
    print("🟢 [CLAP] Copying clap_cache.npz from previous run...")
    shutil.copy2(prev_cache, CLAP_CACHE_PATH)
    clap_cache = dict(np.load(CLAP_CACHE_PATH))
    print(f"🟢 [CLAP] Loaded {len(clap_cache)} embeddings from previous run ✓")

else:
    print(f"\n🟢 [CLAP] Pre-computing embeddings for {len(target_pool)} target IRs...")

    # Need IR audio loaded — if catalogue was loaded from disk, lazy-load now
    if _catalogue_loaded_from_disk:
        print("   (Lazy-loading IR audio for embedding computation...)")
        def _lazy_load_ir(ir_id: str) -> Optional[np.ndarray]:
            if ir_id.startswith('mit_'):
                for f in MIT_IR_DIR.rglob('*.wav'):
                    if f.stem == ir_id[4:]:
                        audio, _ = librosa.load(str(f), sr=SR, mono=True)
                        peak = np.max(np.abs(audio))
                        return (audio / peak).astype(np.float32) if peak > 1e-6 else audio
            elif ir_id.startswith('user_'):
                for ext in ('*.irs', '*.wav'):
                    for f in PATHS['irs'].rglob(ext):
                        if f.stem == ir_id[5:]:
                            audio, _ = librosa.load(str(f), sr=SR, mono=True)
                            peak = np.max(np.abs(audio))
                            return (audio / peak).astype(np.float32) if peak > 1e-6 else audio
            return None

        all_irs = {}
        for ir_id in target_pool + bad_pool:
            a = _lazy_load_ir(ir_id)
            if a is not None:
                all_irs[ir_id] = {'audio': a, 'source': ir_catalogue[ir_id]['source']}

    ref_noise = np.random.randn(SR * 3).astype(np.float32) * 0.1

    clap_cache: Dict[str, np.ndarray] = {}
    for i, ir_id in enumerate(target_pool):
        if ir_id not in all_irs:
            continue
        ir_audio = all_irs[ir_id]['audio']
        scene = fftconvolve(ref_noise, ir_audio, mode='full')[:SR * 3]
        scene = scene / (np.max(np.abs(scene)) + 1e-8)
        clap_cache[ir_id] = get_clap_audio_embedding(scene)
        if (i + 1) % 50 == 0:
            print(f"  {i+1}/{len(target_pool)}")

    np.savez(CLAP_CACHE_PATH, **clap_cache)
    print(f"🟢 [CLAP] Cached {len(clap_cache)} embeddings ✓  (dim={CLAP_DIM})")


🟢 [CLAP] Device: cpu
🟢 [CLAP] Loading frozen model from current working dir...


Loading weights:   0%|          | 0/555 [00:00<?, ?it/s]

🟢 [CLAP] Loaded — embedding dim = 512

🟢 [CLAP] Pre-computing embeddings for 270 target IRs...
  50/270
  100/270
  150/270
  200/270
  250/270
🟢 [CLAP] Cached 270 embeddings ✓  (dim=512)


In [ ]:
import pyloudnorm as pyln

# ═══════════════════════════════════════════════════════════════════════
# PHASE 4 — Vocal Sterilization & "Dead" Audio Guarantee
# ═══════════════════════════════════════════════════════════════════════

meter = pyln.Meter(SR)
STATE_FILE = STERILIZED_DIR / 'sterilize_state.json'

# ─── Check if a previous run already completed sterilization ──────────
prev_sterilized = PREV_RUN_PATH / 'sterilized_batches'
prev_state_file = prev_sterilized / 'sterilize_state.json' if prev_sterilized.exists() else None

_skip_sterilization = False
if prev_state_file and prev_state_file.exists():
    with open(prev_state_file) as f:
        prev_state = json.load(f)
    if prev_state.get('completed', False):
        print("🟢 [Sterilize] Previous run completed sterilization. Copying batches...")
        shutil.copytree(prev_sterilized, STERILIZED_DIR, dirs_exist_ok=True)
        _skip_sterilization = True
        print("🟢 [Sterilize] Sterilized batches copied from previous run ✓")

if STATE_FILE.exists() and not _skip_sterilization:
    with open(STATE_FILE) as f:
        _st = json.load(f)
    if _st.get('completed', False):
        print("🟢 [Sterilize] Already completed in this working dir. Skipping.")
        _skip_sterilization = True

if not _skip_sterilization:
    def discover_audio_files() -> List[Tuple[Path, str]]:
        """Collect all vocal files from all datasets."""
        
        files = []
        # LJSpeech
        for f in sorted(PATHS['ljspeech'].rglob('*.wav')):
            files.append((f, 'ljspeech'))
        # VCTK — subfolder per speaker
        for f in sorted(PATHS['vctk'].rglob('*.wav')):
            files.append((f, f'vctk_{f.parent.name}'))
        # Language Identifier — english clips
        for ext in ('*.wav', '*.mp3', '*.ogg'):
            for f in sorted(PATHS['langid_en'].rglob(ext)):
                files.append((f, 'langid_en'))
        return files

    def sterilize_and_segment(filepath: Path, tag: str) -> List[dict]:
        """Load → noise-reduce → trim → LUFS normalize → segment into 5s windows."""
        try:
            audio, _ = librosa.load(str(filepath), sr=SR, mono=True)
        except Exception:
            return []

        if len(audio) < SR * 1.0:
            return []

        # Spectral noise reduction — strip residual room tone / hiss
        audio = nr.reduce_noise(y=audio, sr=SR, stationary=True, prop_decrease=0.85)

        # Trim absolute silence
        audio, _ = librosa.effects.trim(audio, top_db=40)
        if len(audio) < SR * 1.5:
            return []

        # Normalize loudness to -23 LUFS
        try:
            loudness = meter.integrated_loudness(audio)
            if loudness > -70:
                audio = pyln.normalize.loudness(audio, loudness, -23.0)
        except Exception:
            pass

        # Segment into rigid CLIP_SAMPLES (5.0s) chunks
        segments = []
        for start in range(0, len(audio) - SR, CLIP_SAMPLES):
            chunk = audio[start : start + CLIP_SAMPLES]
            if len(chunk) < CLIP_SAMPLES:
                chunk = np.pad(chunk, (0, CLIP_SAMPLES - len(chunk)))

            rms = np.sqrt(np.mean(chunk ** 2))
            if rms < 1e-4:
                continue

            segments.append({
                'audio':   chunk.astype(np.float32),
                'file':    filepath.name,
                'dataset': tag,
            })
        return segments

    # ── Discover & deterministic shuffle ──
    print("🟢 [Sterilize] Discovering audio files...")
    all_audio_files = discover_audio_files()

    # Sort alphabetically → shuffle with fixed seed for reproducibility
    all_audio_files.sort(key=lambda x: str(x[0]))
    random.Random(42).shuffle(all_audio_files)
    print(f"🟢 [Sterilize] Found {len(all_audio_files)} source files")

    # ── Resume from sterilize checkpoint ──
    cursor_start = 0
    if STATE_FILE.exists():
        with open(STATE_FILE) as f:
            cursor_start = json.load(f).get('cursor', 0)

    STERILIZE_CHUNK = 500
    vocal_segments: List[dict] = []

    print(f"🟢 [Sterilize] Processing from file index {cursor_start}...")
    for i in range(cursor_start, len(all_audio_files)):
        fpath, tag = all_audio_files[i]

        # Budget check
        if get_output_size_gb() > MAX_OUTPUT_GB:
            print(f"\n⚠ Output size limit reached. Saving sterilization progress.")
            break

        if i % 200 == 0 and i > cursor_start:
            print(f"🟢 Processed {i}/{len(all_audio_files)} files → "
                  f"{len(vocal_segments)} segments in RAM buffer")

        segs = sterilize_and_segment(fpath, tag)
        vocal_segments.extend(segs)

        # 💾 Granular disk flushing every STERILIZE_CHUNK files
        if (i + 1) % STERILIZE_CHUNK == 0 or (i + 1) == len(all_audio_files):
            batch_index = (i + 1) // STERILIZE_CHUNK
            batch_path = STERILIZED_DIR / f"sterilized_batch_{batch_index:04d}.pkl"

            with open(batch_path, 'wb') as f:
                pickle.dump(vocal_segments, f)

            completed = (i + 1) >= len(all_audio_files)
            with open(STATE_FILE, 'w') as f:
                json.dump({'cursor': i + 1, 'completed': completed}, f)

            print(f"💾 Saved {len(vocal_segments)} segments to {batch_path.name}. RAM cleared.")
            vocal_segments.clear()

# ── Aggregate: read all .pkl batches to compile the segment count ──
print("\n🟢 [Sterilize] Compiling dataset breakdown from disk...")
ds_counts = defaultdict(int)
total_segments = 0

for batch_file in sorted(STERILIZED_DIR.glob('*.pkl')):
    with open(batch_file, 'rb') as f:
        batch_data = pickle.load(f)
        total_segments += len(batch_data)
        for s in batch_data:
            ds_counts[s['dataset'].split('_')[0]] += 1

print(f"🟢 [Sterilize] Total sterile segments: {total_segments}")
for ds, cnt in sorted(ds_counts.items()):
    print(f"  {ds}: {cnt}")


🟢 [Sterilize] Discovering audio files...
🟢 [Sterilize] Found 133835 source files
🟢 [Sterilize] Processing from file index 0...
🟢 Processed 200/133835 files → 230 segments in RAM buffer
🟢 Processed 400/133835 files → 462 segments in RAM buffer
💾 Saved 579 segments to sterilized_batch_0001.pkl. RAM cleared.
🟢 Processed 600/133835 files → 111 segments in RAM buffer
🟢 Processed 800/133835 files → 340 segments in RAM buffer
💾 Saved 561 segments to sterilized_batch_0002.pkl. RAM cleared.
🟢 Processed 1000/133835 files → 0 segments in RAM buffer
🟢 Processed 1200/133835 files → 219 segments in RAM buffer
🟢 Processed 1400/133835 files → 431 segments in RAM buffer
💾 Saved 541 segments to sterilized_batch_0003.pkl. RAM cleared.
🟢 Processed 1600/133835 files → 108 segments in RAM buffer
🟢 Processed 1800/133835 files → 320 segments in RAM buffer
💾 Saved 543 segments to sterilized_batch_0004.pkl. RAM cleared.
🟢 Processed 2000/133835 files → 0 segments in RAM buffer
🟢 Processed 2200/133835 files → 228

---
> ⚠️ **Phase Skip**: Phases 2–4 above can be skipped entirely if a previous run
> completed them successfully. The notebook detects existing `ir_catalogue.json`,
> `clap_cache.npz`, and `sterilize_state.json` to bypass redundant computation.
> Previous run files are located at `/kaggle/input/notebooks/itorousa/genesis-data-run#`.
---


## 🔗 Checkpoint Chaining (20 GB Limit)

If the output hit the size limit before processing all vocals:

1. **Save this notebook's output** as a Kaggle dataset (e.g. `genesis-data-run1`)
2. **Create a new notebook** (or re-run this one) and attach:
   - All the same input datasets (IRs, LJSpeech, VCTK, Language Identifier)
   - The previous output as input (update `PREV_RUN_PATH` in Cell 3)
3. **Run all cells** — the checkpoint system automatically skips completed work

Each run produces ~19 GB of training triples. Chain as many times as needed.

### Using the Data in Training

```python
# In the training notebook, load all batches from all runs:
import numpy as np
from pathlib import Path

run_dirs = [
    Path('/kaggle/input/genesis-data-run1/batches'),
    Path('/kaggle/input/genesis-data-run2/batches'),
    # ... add more runs
]

for run_dir in run_dirs:
    for batch_file in sorted(run_dir.glob('batch_*.npz')):
        data = np.load(batch_file)
        source_audio = data['source_audio']   # (N, 240000) int16
        target_audio = data['target_audio']   # (N, 240000) int16
        target_clap  = data['target_clap']    # (N, CLAP_DIM) float32
        # Convert int16 back to float32: audio = source_audio.astype(np.float32) / 32767
        # Compute STFT on-the-fly during training for memory efficiency
```
